### Study of the effect of national factors on home prices in the US


**Task:** Using publicly available data on the national factors that impact the supply and demand of homes in the US, build a data science model to study the effect of these variables on home prices.

**Approach:** The following variables are chosen for the study:

1. Unemployment Rate
2. Employment Rate
3. Per capita GDP
4. Median Household Income
5. Construction Prices
6. CPI
7. Interest Rates
8. The number of new houses supplied
9. Working Population
10. Urban Population
11. Percentage of population above 65
12. Housing subsidies
13. Number of Households

As a proxy for home prices, the S&P **Case-Shiller Index** is used.

**Note:** Most of the data is downloaded from [https://fred.stlouisfed.org/].

Data for all the variables is downloaded, preprocessed, and combined to create a dataset using the **Extract Transform Load (ETL)** method. Data for different variables had different frequencies. So, to combine the data, the necessary interpolations are made.


#### Importing neccessary libraries

In [1]:
import numpy as np
import pandas as pd

#### Perform ETL

In [2]:
# Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv("prepared_dataset.csv")

# Changing dtype of date column
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])

# Selecting data till JULY 2023
mask = df_CS["DATE"] <= "2023-07-01"
df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print("Shape of the CASE-SHILLER Index:- ", df_CS.shape)
df_CS.tail()


Shape of the CASE-SHILLER Index:-  (252, 17)


,DATE,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
247,2022-08-01,301.596,2022,8,65569.000000,2.073941e+08,8.7,295.320,3.7,71.466900,342.753,2.33,83.1,131202.0,74580.0,48.021,17.1
248,2022-09-01,299.380,2022,9,65676.000000,2.074743e+08,9.7,296.539,3.5,71.444231,336.464,2.56,83.1,131202.0,74580.0,48.021,17.1
249,2022-10-01,298.922,2022,10,65783.000000,2.074563e+08,9.7,297.987,3.7,71.247799,333.796,3.08,83.1,131202.0,74580.0,48.021,17.1
250,2022-11-01,298.312,2022,11,65881.333333,2.075097e+08,9.4,298.598,3.6,71.323036,330.369,3.78,83.1,131202.0,74580.0,48.021,17.1
251,2022-12-01,297.471,2022,12,65979.666667,2.075066e+08,8.5,298.990,3.5,71.575987,326.449,4.10,83.1,131202.0,74580.0,48.021,17.1


In [3]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("UNRATE.csv")
df_unemp.drop([259], inplace = True)
print("Unemployment Rate Data:- ", df_unemp.shape)
df_unemp.tail()

FileNotFoundError: [Errno 2] No such file or directory: 'UNRATE.csv'

In [ ]:
# Reading Employment Rate Data into a dataframe
df_emp = pd.read_csv("EMPRATE.csv")
df_emp = df_emp.rename(columns={'LREM64TTUSM156S': 'EmpRate'})
df_emp.drop([259], inplace = True)
print("shape of the Employment Rate Data:- ", df_emp.shape)
df_emp.tail()

In [ ]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("GDP_per_capita.csv", names = ["DATE", "A939RX0Q048SBEA"], skiprows = 1)
df_pcgdp = df_pcgdp.rename(columns={'A939RX0Q048SBEA': 'Per_Capita_GDP'})
print("Shape of the Per Capita GDP Data:- ", df_pcgdp.shape)
df_pcgdp.tail()

The data is quarterly. We will impute for other months using linear interpolation after we create the final dataframe combining all the data.


In [ ]:
# Interest Rate Data
df_Fed_rate = pd.read_csv("FEDFUNDS.csv").drop([259])
print("Shape of the Interest rate data:- ",df_Fed_rate.shape)
df_Fed_rate.tail()

In [ ]:
# Reading Construction Material Data into a dataframe
df_cons_price_index = pd.read_csv("construction_price_ppi.csv", names = ["DATE", "WPUSI012011"], skiprows = 1)
df_cons_price_index = df_cons_price_index.rename(columns={'WPUSI012011': 'Cons_Material'})
df_cons_price_index.drop([259], inplace = True)
print("Shape of the Construction Material Data:- ", df_cons_price_index.shape)
df_cons_price_index.tail()

In [ ]:
# Consumer Price Index
df_CPI = pd.read_csv("CPIAUCSL.csv", names = ["DATE", "CPIAUCSL"], skiprows = 1).drop([259])
df_CPI = df_CPI.rename(columns={'CPIAUCSL': 'CPI'})
print("Shape of the Consumer Price Index:- ", df_CPI.shape)
df_CPI.tail()

In [ ]:
# Monthly new house supply
df_house = pd.read_csv("monthly_house_supply.csv", names = ["DATE", "MSACSR"], skiprows = 1).drop([259])
df_house = df_house.rename(columns={'MSACSR': 'Houses'})
print("Shape of the monthly house supply data:- ", df_house.shape)
df_house.tail()


In [ ]:
# Population above 65

df_oldpop = pd.read_csv("old_age_pop.csv", names = ["DATE", "old_age_pop"], skiprows = 1)
df_oldpop['DATE'] = pd.to_datetime(df_oldpop['DATE'], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")
print("Shape of the population data age above 65:- ", df_oldpop.shape)
df_oldpop.tail()

In [ ]:
# Urban Population Percent

df_urban = pd.read_csv("urban_pop.csv")
df_urban['DATE'] = pd.to_datetime(df_urban['DATE'], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")
print("Shape of the urban population percent data:- ", df_urban.shape)
df_urban.tail()

In [ ]:
# Housing Subsidies

df_subsidy = pd.read_csv("housing_subsidies.csv", names = ["DATE", "Subsidy"], skiprows = 1)
print("Shape of the housing subsidies:- ", df_subsidy.shape)
df_subsidy.tail()


In [ ]:
# Working age population

df_working = pd.read_csv("working_age_population.csv", names = ["DATE", "LFWA64TTUSM647S"], skiprows = 1).drop([259])
df_working = df_working.rename(columns={'LFWA64TTUSM647S': 'working_age_pop'})
print("Shape of the working age population:- ", df_working.shape)
df_working.tail()

In [ ]:
# Real Median Household Income

df_income = pd.read_csv("median_household_income.csv", names = ["DATE", "MEHOINUSA672N"], skiprows = 1)
df_income = df_income.rename(columns={'MEHOINUSA672N': 'median_income'})
print("Shape of the median household income data:- ", df_income.shape)
df_income.tail()


In [ ]:
# Total number of households

df_households = pd.read_csv("household.csv", names = ["DATE", "TTLHH"], skiprows = 1)
df_households = df_households.rename(columns={'TTLHH': 'Num_Households'})
print("Shape of the total households data:- ", df_households.shape)
df_households.tail()


In [ ]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df_CS.head()


In [ ]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_CS, df_working, df_house, df_CPI, df_unemp, df_emp, df_cons_price_index, df_Fed_rate]
for df1 in df_bymonth:
    df1["DATE"] = pd.to_datetime(df1["DATE"])
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.head()


In [ ]:
# Merging other dataframes 
others = [df_urban, df_households, df_income, df_subsidy, df_oldpop]
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()

In [ ]:
print(df.shape)

Check missing values (NAN)

In [ ]:
df.isna().sum()

The "Per_Capita_GDP" column has missing values because the data was quarterly. The missing values in the other columns are due to the unavailability of fresh data. We will first fill in the missing values in the "Per_Capita_GDP" column using linear interpolation. We will drop the rows with missing values in the other columns. This means that we will use data from 2002 to 2022.

**Interpolation:**

Interpolation is a mathematical technique used to estimate values that are missing in a dataset based on the values of neighboring data points. It calculates intermediate values based on the existing data.



In [ ]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()

In [ ]:
df

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.isna().sum()

In [ ]:
df

In [ ]:
print("Shape of the dataframe after preprocessing:- ", df.shape)

This is our preprocessed datset. Let's save it as "prepared_dataset.csv".


In [ ]:
df.to_csv("prepared_dataset.csv")

In [ ]:
us_house_price_df = pd.read_csv("prepared_dataset.csv").set_index("DATE")
us_house_price_df.head()

## To be continued...........